In [1]:
import pandas as pd
import numpy as np
df = pd.read_csv('RedditTokenized.csv')
df.head()

,comment,hate_speech,tokens,tokens_spacy,word2vec_vector,cleaned_comment,fasttext_vector
0,A subsection of retarded Hungarians? Oh boy. b...,1,"['a', 'subsection', 'of', 'retarded', 'hungari...","['a', 'subsection', 'of', 'retarded', 'hungari...",[ 0.36349716 -0.22094536 0.01051669 -0.602767...,a subsection of retarded hungarians oh boy bra...,"[0.004770048428326845, -0.03953176364302635, -..."
1,Iii. Just got off work. Foundation and groundi...,0,"['iii', '.', 'just', 'got', 'off', 'work', '.'...","['iii', '.', 'just', 'got', 'off', 'work', '.'...",[ 0.06504157 -0.12219669 0.10038512 -0.005708...,iii just got off work foundation and grounding...,"[-0.012044071219861507, -0.011684201657772064,..."
2,wow i guess cowboys are the same in every country,0,"['wow', 'i', 'guess', 'cowboys', 'are', 'the',...","['wow', 'i', 'guess', 'cowboys', 'are', 'the',...",[ 2.92166289e-01 3.85734248e-01 -4.80327269e-...,wow i guess cowboys are the same in every country,"[-0.0008766286191530526, 0.0192383024841547, 0..."
3,Owen Benjamin's cowboy song goes for every cou...,0,"['owen', 'benjamin', ""'s"", 'cowboy', 'song', '...","['owen', 'benjamin', ""'s"", 'cowboy', 'song', '...",[ 0.07474209 0.08315161 -0.081609 -0.397689...,owen benjamins cowboy song goes for every coun...,"[0.00418263953179121, -0.0029789397958666086, ..."
4,"> ""y'all hear sun?"" by all means I live in a s...",0,"['>', '``', ""y'all"", 'hear', 'sun', '?', ""''"",...","['>', '""', ""y'"", 'all', 'hear', 'sun', '?', '""...",[ 0.19263356 -0.10749349 -0.08967624 -0.271426...,yall hear sun by all means i live in a small t...,"[-6.847345503047109e-05, 0.0004209601611364633..."


## Improving model

#### Techniques for Handling Imbalanced Data
- ##### Data Resampling Techniques

In [2]:
#Over-sampling the minority class

from imblearn.over_sampling import SMOTE

# Convert string representation of vectors to actual numerical format
df['word2vec_vector'] = df['word2vec_vector'].apply(lambda x: np.fromstring(x.strip('[]'), sep=','))
df['fasttext_vector'] = df['fasttext_vector'].apply(lambda x: np.fromstring(x.strip('[]'), sep=','))

# Combine numerical features
X = np.vstack(df['word2vec_vector'].values)
# Alternatively, you can also use 'fasttext_vector' or combine both
# X = np.hstack((np.vstack(df['word2vec_vector'].values), np.vstack(df['fasttext_vector'].values)))

y = df['hate_speech']

# Initialize SMOTE
smote = SMOTE(random_state=42)

# Fit and resample
X_resampled, y_resampled = smote.fit_resample(X, y)

# Create DataFrame from resampled data
df_resampled = pd.DataFrame(X_resampled, columns=[f'vector_{i}' for i in range(X_resampled.shape[1])])
df_resampled['hate_speech'] = y_resampled

# Optionally, recombine vectors into a format similar to the original
df_resampled['word2vec_vector'] = df_resampled[[f'vector_{i}' for i in range(X_resampled.shape[1])]].apply(lambda x: x.values, axis=1)


C:\Users\Aman Shekhar Sachan\AppData\Local\Temp\ipykernel_8368\2969848702.py:6: DeprecationWarning: string or file could not be read to its end due to unmatched data; this will raise a ValueError in the future.
  df['word2vec_vector'] = df['word2vec_vector'].apply(lambda x: np.fromstring(x.strip('[]'), sep=','))


In [3]:
#Undersampling the Majority Class

from imblearn.under_sampling import RandomUnderSampler

# Extract the feature vectors and target
X = np.vstack(df['word2vec_vector'].values)
y = df['hate_speech']

# Apply undersampling
undersampler = RandomUnderSampler(random_state=42)
X_resampled, y_resampled = undersampler.fit_resample(X, y)

# Reconstruct DataFrame
df_resampled = pd.DataFrame(X_resampled, columns=[f'vector_{i}' for i in range(X_resampled.shape[1])])
df_resampled['hate_speech'] = y_resampled

# Optionally, recombine vectors into a format similar to the original
df_resampled['word2vec_vector'] = df_resampled[[f'vector_{i}' for i in range(X_resampled.shape[1])]].apply(lambda x: x.values, axis=1)


In [4]:
# Combination of Over- and Undersampling

from imblearn.combine import SMOTETomek
#SMOTETomek
smote_tomek = SMOTETomek(random_state=42)
X_resampled_tomek, y_resampled_tomek = smote_tomek.fit_resample(X, y)

# Reconstruct DataFrame for SMOTETomek
df_resampled_tomek = pd.DataFrame(X_resampled_tomek, columns=[f'vector_{i}' for i in range(X_resampled_tomek.shape[1])])
df_resampled_tomek['hate_speech'] = y_resampled_tomek

# Optionally, recombine vectors into a format similar to the original
df_resampled_tomek['word2vec_vector'] = df_resampled_tomek[[f'vector_{i}' for i in range(X_resampled_tomek.shape[1])]].apply(lambda x: x.values, axis=1)


In [5]:
# Using Class Weights

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# Assuming you have your dataset ready in X and y
# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Logistic Regression with class weights
model = LogisticRegression(class_weight='balanced', random_state=42)
model.fit(X_train, y_train)

# Predictions and evaluation
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.75      0.45      0.56      3341
           1       0.25      0.54      0.34      1109

    accuracy                           0.48      4450
   macro avg       0.50      0.50      0.45      4450
weighted avg       0.62      0.48      0.51      4450



In [6]:
#Algorithmic Approaches
#Many algorithms can inherently handle imbalanced data by adjusting for class weights.

#Example with RandomForest

from sklearn.ensemble import RandomForestClassifier

# RandomForest with class weights
rf_model = RandomForestClassifier(class_weight='balanced', random_state=42)
rf_model.fit(X_train, y_train)

# Predictions and evaluation
y_pred_rf = rf_model.predict(X_test)
print(classification_report(y_test, y_pred_rf))


              precision    recall  f1-score   support

           0       0.75      0.76      0.75      3341
           1       0.25      0.25      0.25      1109

    accuracy                           0.63      4450
   macro avg       0.50      0.50      0.50      4450
weighted avg       0.63      0.63      0.63      4450



In [7]:
#Anomaly Detection Techniques
#Anomaly detection can be useful for imbalanced datasets, particularly when the minority class is significantly underrepresented.

#Example with Isolation Forest

from sklearn.ensemble import IsolationForest

# Fit Isolation Forest
iso_forest = IsolationForest(contamination=0.1, random_state=42)
y_train_pred = iso_forest.fit_predict(X_train)
y_test_pred = iso_forest.predict(X_test)

# Convert the output to binary classification (1 for normal, -1 for anomaly)
y_train_pred = [1 if x == -1 else 0 for x in y_train_pred]
y_test_pred = [1 if x == -1 else 0 for x in y_test_pred]

print(classification_report(y_test, y_test_pred))


              precision    recall  f1-score   support

           0       0.75      0.90      0.82      3341
           1       0.24      0.10      0.14      1109

    accuracy                           0.70      4450
   macro avg       0.49      0.50      0.48      4450
weighted avg       0.62      0.70      0.65      4450



In [8]:
#Generating Synthetic Data
#Using techniques like SMOTE (Synthetic Minority Over-sampling Technique) to generate synthetic samples for the minority class can help balance the dataset.

#Example with SMOTE

from imblearn.over_sampling import SMOTE

# SMOTE
smote = SMOTE(random_state=42)
X_res, y_res = smote.fit_resample(X, y)

# Splitting resampled data
X_train_res, X_test_res, y_train_res, y_test_res = train_test_split(X_res, y_res, test_size=0.2, random_state=42)

# Model training with resampled data
model_resampled = LogisticRegression(random_state=42)
model_resampled.fit(X_train_res, y_train_res)

# Predictions and evaluation
y_pred_resampled = model_resampled.predict(X_test_res)
print(classification_report(y_test_res, y_pred_resampled))


              precision    recall  f1-score   support

           0       0.49      0.47      0.48      3379
           1       0.50      0.52      0.51      3405

    accuracy                           0.49      6784
   macro avg       0.49      0.49      0.49      6784
weighted avg       0.49      0.49      0.49      6784



#### Evaluation and Validation
##### Proper evaluation is crucial for imbalanced datasets. Metrics like Precision, Recall, F1-score, and AUC-ROC should be used.

In [9]:
from sklearn.metrics import roc_auc_score, precision_recall_curve, auc, confusion_matrix

# Confusion Matrix
conf_matrix = confusion_matrix(y_test, y_pred_rf)
print("Confusion Matrix:\n", conf_matrix)

# Precision-Recall Curve
precision, recall, thresholds = precision_recall_curve(y_test, y_pred_rf)
pr_auc = auc(recall, precision)
print(f"Precision-Recall AUC: {pr_auc}")

# ROC-AUC Score
roc_auc = roc_auc_score(y_test, y_pred_rf)
print(f"ROC-AUC Score: {roc_auc}")


Confusion Matrix:
 [[2525  816]
 [ 836  273]]
Precision-Recall AUC: 0.342360796219475
ROC-AUC Score: 0.5009647333225555


#### Pipeline Implementation

In [10]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from imblearn.pipeline import Pipeline as ImbPipeline
from imblearn.over_sampling import SMOTE

# Create a pipeline
pipeline = ImbPipeline([
    ('scaler', StandardScaler()),
    ('smote', SMOTE(random_state=42)),
    ('classifier', LogisticRegression(random_state=42))
])

# Fit the pipeline
pipeline.fit(X_train, y_train)

# Predictions and evaluation
y_pred_pipeline = pipeline.predict(X_test)
print(classification_report(y_test, y_pred_pipeline))


              precision    recall  f1-score   support

           0       0.75      0.45      0.56      3341
           1       0.25      0.54      0.34      1109

    accuracy                           0.48      4450
   macro avg       0.50      0.50      0.45      4450
weighted avg       0.62      0.48      0.51      4450



### The results indicate that the model is not performing well:

- ##### High False Positives and False Negatives:  The high number of both false positives and false negatives suggests that the model is not correctly identifying the minority class (hate speech).
- ##### Low Precision-Recall AUC:  Indicates poor balance between precision and recall.
- ##### ROC-AUC close to 0.5:  Shows the model struggles to distinguish between classes.

In [11]:
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, roc_auc_score, precision_recall_curve, auc, confusion_matrix
from imblearn.over_sampling import SMOTE
from sklearn.linear_model import LogisticRegression

# Assuming X contains FastText vectors and y is the target labels
X = np.array(df['fasttext_vector'].tolist())  # Use FastText vectors
y = df['hate_speech']  # Target labels

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Resample the data
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_train, y_train)

# Logistic Regression with hyperparameter tuning
param_grid = {
    'C': [0.1, 1, 10, 100],
    'solver': ['lbfgs', 'liblinear']
}
model = GridSearchCV(LogisticRegression(class_weight='balanced', max_iter=1000), param_grid, cv=5, scoring='f1')
model.fit(X_resampled, y_resampled)

# Predictions and evaluation
y_pred = model.predict(X_test)

# Confusion Matrix
conf_matrix = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:\n", conf_matrix)

# Precision-Recall Curve
precision, recall, thresholds = precision_recall_curve(y_test, y_pred)
pr_auc = auc(recall, precision)
print(f"Precision-Recall AUC: {pr_auc}")

# ROC-AUC Score
roc_auc = roc_auc_score(y_test, y_pred)
print(f"ROC-AUC Score: {roc_auc}")

# Detailed report
print(classification_report(y_test, y_pred))


Confusion Matrix:
 [[2524  817]
 [ 328  781]]
Precision-Recall AUC: 0.6333409186838915
ROC-AUC Score: 0.7298502443478286
              precision    recall  f1-score   support

           0       0.88      0.76      0.82      3341
           1       0.49      0.70      0.58      1109

    accuracy                           0.74      4450
   macro avg       0.69      0.73      0.70      4450
weighted avg       0.79      0.74      0.76      4450



In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Assuming the structure of your dataset
X = df['cleaned_comment'].astype(str)  # Convert to string explicitly if needed
y = df['hate_speech']  # Replace with your actual target column

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Perform feature extraction using TfidfVectorizer
vectorizer = TfidfVectorizer(max_features=1000)
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

# Resample the training data using SMOTE due to class imbalance
smote = SMOTE(random_state=42)
X_train_tfidf_resampled, y_train_resampled = smote.fit_resample(X_train_tfidf, y_train)

# Logistic Regression with hyperparameter tuning using GridSearchCV
param_grid = {
    'C': [0.01, 0.1, 1, 10, 100],
    'solver': ['lbfgs', 'liblinear']
}
model = GridSearchCV(LogisticRegression(class_weight='balanced', max_iter=1000), param_grid, cv=5, scoring='f1')
model.fit(X_train_tfidf_resampled, y_train_resampled)

# Predictions on the test set
y_pred = model.predict(X_test_tfidf)

# Evaluate the model
conf_matrix = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:\n", conf_matrix)

precision, recall, thresholds = precision_recall_curve(y_test, y_pred)
pr_auc = auc(recall, precision)
print(f"Precision-Recall AUC: {pr_auc}")

roc_auc = roc_auc_score(y_test, y_pred)
print(f"ROC-AUC Score: {roc_auc}")

print(classification_report(y_test, y_pred))


Confusion Matrix:
 [[2779  562]
 [ 311  798]]
Precision-Recall AUC: 0.6881097619846513
ROC-AUC Score: 0.7756770338950801
              precision    recall  f1-score   support

           0       0.90      0.83      0.86      3341
           1       0.59      0.72      0.65      1109

    accuracy                           0.80      4450
   macro avg       0.74      0.78      0.76      4450
weighted avg       0.82      0.80      0.81      4450



#### Great job on improving the model performance! Now that we have these evaluation metrics:

- ##### Precision-Recall AUC: 0.688
- ##### ROC-AUC Score: 0.776
- ##### F1-score: 0.65 for class 1 (hate speech)

### Next steps to consider for further improvement:

#### Random Forest Classifier

In [13]:
from sklearn.ensemble import RandomForestClassifier

# Initialize Random Forest classifier
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)

# Fit the model
rf_model.fit(X_train_tfidf_resampled, y_train_resampled)

# Predictions on the test set
y_pred_rf = rf_model.predict(X_test_tfidf)

# Evaluate the model
print("Random Forest Classifier Performance:")
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_rf))
print("Precision-Recall AUC:", auc(recall, precision_recall_curve(y_test, y_pred_rf)[0]))
print("ROC-AUC Score:", roc_auc_score(y_test, y_pred_rf))
print(classification_report(y_test, y_pred_rf))


Random Forest Classifier Performance:
Confusion Matrix:
 [[3179  162]
 [ 354  755]]
Precision-Recall AUC: 0.8063958932310429
ROC-AUC Score: 0.816152515580261
              precision    recall  f1-score   support

           0       0.90      0.95      0.92      3341
           1       0.82      0.68      0.75      1109

    accuracy                           0.88      4450
   macro avg       0.86      0.82      0.84      4450
weighted avg       0.88      0.88      0.88      4450



#### Using Multinomial Naive Bayes for Hate Speech Detection

In [14]:
# Import necessary libraries
from sklearn.naive_bayes import MultinomialNB

# Handle missing values in the text data
# Option 1: Remove rows with NaN values in 'cleaned_comment'
df = df.dropna(subset=['cleaned_comment'])

# Option 2: Fill NaN values with a placeholder (if appropriate)
# df['cleaned_comment'] = df['cleaned_comment'].fillna('missing')

# Extract features and labels
X = df['cleaned_comment'].astype(str)  # Convert to string to avoid issues with TfidfVectorizer
y = df['hate_speech']

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Feature extraction using TfidfVectorizer
vectorizer = TfidfVectorizer(max_features=1000, stop_words='english')
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

# Train the Naive Bayes classifier
naive_bayes = MultinomialNB()
naive_bayes.fit(X_train_tfidf, y_train)

# Predictions on the test set
y_pred = naive_bayes.predict(X_test_tfidf)
y_pred_proba = naive_bayes.predict_proba(X_test_tfidf)[:, 1]

# Evaluate the model
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

# ROC-AUC Score
roc_auc = roc_auc_score(y_test, y_pred_proba)
print(f"ROC-AUC Score: {roc_auc}")

# Precision-Recall Curve and AUC
precision, recall, thresholds = precision_recall_curve(y_test, y_pred_proba)
pr_auc = auc(recall, precision)
print(f"Precision-Recall AUC: {pr_auc}")

Confusion Matrix:
 [[3356   29]
 [ 771  287]]
Classification Report:
               precision    recall  f1-score   support

           0       0.81      0.99      0.89      3385
           1       0.91      0.27      0.42      1058

    accuracy                           0.82      4443
   macro avg       0.86      0.63      0.66      4443
weighted avg       0.84      0.82      0.78      4443

ROC-AUC Score: 0.8432351947460859
Precision-Recall AUC: 0.7221803482578294


### Observations
- ##### The Random Forest Classifier has a high overall accuracy (0.88) and good precision and recall for both classes, especially class 0.
- ##### The "Other Model" has a very high precision for class 1 but a very low recall, indicating that it misses many of the true positives for class 1.
- ##### The Random Forest Classifier provides a better balance between precision and recall for the minority class, which is crucial for handling class imbalance.
- ##### The ROC-AUC score of 0.816 and Precision-Recall AUC of 0.806 for the Random Forest Classifier suggest that it is relatively robust and balanced.

#### Simplified Hyperparameter Tuning with RandomizedSearchCV


In [15]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import confusion_matrix, classification_report
import numpy as np

# Define the parameter grid for Random Forest
'''
param_dist_rf = {
    'n_estimators': [100, 200, 300, 400, 500],
    'max_depth': [5, 10, 15, 20, 25, 30, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'bootstrap': [True, False]
}
'''

param_dist_rf = {
    'n_estimators': [80, 100, 120],
    'max_depth': [None, 10, 15],
    'min_samples_split': [4, 5, 6],
    'min_samples_leaf': [1, 2, 3],
    'bootstrap': [False]
}


# Initialize RandomizedSearchCV
random_search_rf = RandomizedSearchCV(
    estimator=RandomForestClassifier(class_weight='balanced', random_state=42),
    param_distributions=param_dist_rf,
    n_iter=20,  # Number of parameter settings to try
    cv=5,  # 5-fold cross-validation
    scoring='f1',
    verbose=1,
    random_state=42,
    n_jobs=-1  # Use all available cores
)

# Fit the model and check if fit_transform is executed correctly
try:
    random_search_rf.fit(X_train_tfidf_resampled, y_train_resampled)
except ValueError as e:
    print(f"ValueError during model fitting: {e}")
    exit(1)  # Exit if there's an error

# Best parameters and model
print("Best Parameters:", random_search_rf.best_params_)
best_rf_model = random_search_rf.best_estimator_

# Predictions on the test set
y_pred_rf = best_rf_model.predict(X_test_tfidf)

# Evaluate the model
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_rf))
print("Classification Report:\n", classification_report(y_test, y_pred_rf))


Fitting 5 folds for each of 20 candidates, totalling 100 fits
Best Parameters: {'n_estimators': 80, 'min_samples_split': 4, 'min_samples_leaf': 1, 'max_depth': None, 'bootstrap': False}
Confusion Matrix:
 [[2792  593]
 [ 762  296]]
Classification Report:
               precision    recall  f1-score   support

           0       0.79      0.82      0.80      3385
           1       0.33      0.28      0.30      1058

    accuracy                           0.70      4443
   macro avg       0.56      0.55      0.55      4443
weighted avg       0.68      0.70      0.69      4443



#### Feature Engineering and Selection
##### Enhance your features to potentially improve model performance:

In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.decomposition import TruncatedSVD

# Define the pipeline with TfidfVectorizer and TruncatedSVD
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(max_features=1000, stop_words='english')),
    ('svd', TruncatedSVD(n_components=300, random_state=42)),  # Reduce to 300 features
])

# Transform the text data
X_train_tfidf = pipeline.fit_transform(X_train)
X_test_tfidf = pipeline.transform(X_test)

# Use the transformed data for training and evaluation


#### Threshold Tuning
##### Optimize the decision threshold to balance precision and recall
#### Imbalanced Data Handling
##### Ensure that your data handling techniques are effective in addressing class imbalance:

In [17]:
# Drop any rows with missing 'cleaned_comment' or 'hate_speech'
df = df.dropna(subset=['cleaned_comment', 'hate_speech'])

# Split the dataset into features and labels
X = df['cleaned_comment'].values
y = df['hate_speech'].values

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Feature extraction using TfidfVectorizer
vectorizer = TfidfVectorizer(max_features=1000, stop_words='english')
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

# Handle class imbalance with SMOTE
smote = SMOTE(random_state=42)
X_train_tfidf_resampled, y_train_resampled = smote.fit_resample(X_train_tfidf, y_train)

# Define and train the Random Forest model
rf_model = RandomForestClassifier(class_weight='balanced', random_state=42)
rf_model.fit(X_train_tfidf_resampled, y_train_resampled)

# Make predictions on the test set
y_proba_rf = rf_model.predict_proba(X_test_tfidf)[:, 1]

# Calculate precision-recall pairs
precision, recall, thresholds = precision_recall_curve(y_test, y_proba_rf)

# Find the threshold with the best trade-off
f1_scores = 2 * (precision * recall) / (precision + recall)
best_threshold_index = np.argmax(f1_scores)
best_threshold = thresholds[best_threshold_index]

# Apply the best threshold
y_pred_best_threshold = (y_proba_rf >= best_threshold).astype(int)

# Evaluate the model
print("Confusion Matrix with best threshold:\n", confusion_matrix(y_test, y_pred_best_threshold))
print("Classification Report with best threshold:\n", classification_report(y_test, y_pred_best_threshold))

# Display the best threshold for reference
print("Best Threshold:", best_threshold)


Confusion Matrix with best threshold:
 [[3179  206]
 [ 265  793]]
Classification Report with best threshold:
               precision    recall  f1-score   support

           0       0.92      0.94      0.93      3385
           1       0.79      0.75      0.77      1058

    accuracy                           0.89      4443
   macro avg       0.86      0.84      0.85      4443
weighted avg       0.89      0.89      0.89      4443

Best Threshold: 0.5573443223443223
